# Domain Name Suggestion Generator using IsraaH Model (Chat Template Version)

This notebook demonstrates how to use the specialized domain name suggestion model `IsraaH/domain-name-suggestion-generator` on Google Colab with GPU acceleration, following the chat template approach shown in the official example.

## 1. Setup Environment

First, let's install the required dependencies:

In [ ]:
!pip install -q torch transformers accelerate

## 2. Check GPU Availability

Let's check if we have access to a GPU:

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

## 3. Load the Domain Name Suggestion Model

We'll use the specialized `IsraaH/domain-name-suggestion-generator` model which is already fine-tuned for domain name suggestions:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer and model
model_id = "IsraaH/domain-name-suggestion-generator"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
-)

## 4. Define Domain Suggestion Function

Let's create a function that uses the chat template approach to generate domain suggestions:

In [ ]:
def generate_domain_suggestions(business_description, num_suggestions=5):
    """
    Generate domain name suggestions for a business description using the chat template approach.
    """
    # Create a conversation with the business description
    messages = [
        {"role": "user", "content": f"Generate {num_suggestions} domain name suggestions for: {business_description}"}
    ]
    
    # Apply chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode the response
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    return response

## 5. Test the Domain Suggestion System

Let's test our domain suggestion system with some example business descriptions:

In [ ]:
# Example business descriptions
test_businesses = [
    "A bakery that specializes in artisanal sourdough bread and pastries",
    "An online platform for learning coding and software development skills",
    "A sustainable fashion brand that creates eco-friendly clothing from recycled materials",
    "A mobile app for tracking fitness goals and connecting with personal trainers"
-]

# Generate suggestions for each business
for i, business in enumerate(test_businesses, 1):
    print(f"\n{i}. Business: {business}\n")
    suggestions = generate_domain_suggestions(business)
    print(f"Domain Suggestions:\n{suggestions}\n")
    print("-" * 80)

## 6. Alternative Approach Using Pipeline

Let's also try using the pipeline approach as shown in the official example:

In [ ]:
from transformers import pipeline

# Create a text generation pipeline
pipe = pipeline(
    "text-generation", 
    model="IsraaH/domain-name-suggestion-generator",
    torch_dtype=torch.float16,
    device_map="auto"
-)

In [ ]:
def generate_domain_suggestions_pipeline(business_description, num_suggestions=5):
    """
    Generate domain name suggestions using the pipeline approach.
    """
    messages = [
        {"role": "user", "content": f"Generate {num_suggestions} domain name suggestions for: {business_description}"}
    ]
    
    # Generate response
    outputs = pipe(
        messages,
        max_new_tokens=300,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    
    return outputs[0]["generated_text"]["content"]

## 7. Testing the Pipeline Approach

Let's test our pipeline-based implementation:

In [ ]:
# Test the pipeline version with one business
print("Testing pipeline approach...")
business = test_businesses[0]
print(f"\nBusiness: {business}\n")
suggestions = generate_domain_suggestions_pipeline(business)
print(f"Domain Suggestions:\n{suggestions}\n")
print("-" * 80)

## 8. Batch Processing Implementation

Let's implement batch processing for multiple businesses using the direct model approach:

In [ ]:
def generate_domain_suggestions_batch(business_descriptions, num_suggestions=5):
    """
    Generate domain name suggestions for multiple business descriptions using batch processing.
    """
    # Create conversations for all business descriptions
    all_messages = []
    for business in business_descriptions:
        messages = [
            {"role": "user", "content": f"Generate {num_suggestions} domain name suggestions for: {business}"}
        ]
        all_messages.append(messages)
    
    # Apply chat template to all conversations
    all_inputs = []
    for messages in all_messages:
        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt",
        ).to(model.device)
        all_inputs.append(inputs)
    
    # For simplicity, we'll process each one individually in this example
    # True batch processing would require padding and more complex handling
    responses = []
    for inputs in all_inputs:
        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=300,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )
        
        # Decode the response
        response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        responses.append(response)
    
    return responses

## 9. Testing Batch Processing

Let's test our batch processing implementation:

In [ ]:
# Test batch processing
print("Testing batch processing...")
batch_responses = generate_domain_suggestions_batch(test_businesses)

for i, (business, response) in enumerate(zip(test_businesses, batch_responses), 1):
    print(f"\n{i}. Business: {business}\n")
    print(f"Domain Suggestions:\n{response}\n")
    print("-" * 80)

## 10. Structured Output Generation

Let's try to get more structured output from the model by modifying our prompt:

In [ ]:
def generate_structured_domain_suggestions(business_description, num_suggestions=5):
    """
    Generate domain name suggestions in a more structured format.
    """
    # Create a conversation with a more specific prompt for structured output
    messages = [
        {"role": "user", "content": f"Generate {num_suggestions} domain name suggestions for: {business_description}. Format each suggestion as 'domain.com (confidence: 0.XX)' on a new line."}
    ]
    
    # Apply chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to(model.device)

    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )

    # Decode the response
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    return response

In [ ]:
# Test structured output
print("Testing structured output...")
business = test_businesses[0]
print(f"\nBusiness: {business}\n")
suggestions = generate_structured_domain_suggestions(business)
print(f"Domain Suggestions:\n{suggestions}\n")
print("-" * 80)

## 11. Conclusion

This notebook demonstrated how to:
1. Run the specialized `IsraaH/domain-name-suggestion-generator` model on Google Colab with GPU acceleration
2. Use both the direct model loading approach and the pipeline approach
3. Generate domain name suggestions using the chat template format
4. Implement batch processing for multiple businesses
5. Format the output in a more structured way

Key advantages of this approach:
1. **No Fine-tuning Required**: The model is already specialized for domain name suggestions
2. **Chat Template Compatibility**: Uses the modern chat template approach for better conversation handling
3. **Multiple Implementation Options**: Shows both direct model loading and pipeline approaches
4. **Batch Processing Support**: Implemented for handling multiple requests efficiently
5. **Structured Output Options**: Demonstrates how to format the output for easier parsing

To use this in production, you would want to:
1. Implement additional post-processing to ensure domain name validity
2. Add a database of already registered domains to filter out unavailable names
3. Implement rate limiting and caching for better performance
4. Add safety filters to prevent generation of inappropriate domain names